In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatText, Select
from IPython.display import clear_output
import seaborn as sns

from scipy.integrate import solve_ivp


In [22]:

# Harmonic potential (standard)
def harmonic_potential(x, k=1.0):
    potential = 0.5 * k * x**2
    force = -k * x
    return potential, force




def force_function(x, k, a):
    
    force = -2 * k * x * (x**2 - a)
    return force

def double_force_potential_function(x, k, a):
    potential = 0.5 * k * (x**2 - a)**2
    return potential



def dual_harmonic_potential(x, k=1.0, a = 1.0):
    
    potential = double_force_potential_function(x,k,a)
    force = force_function(x, k,a)
    
    return potential, force


# Simulate particle motion
def simulate_motion(potential_func, params, x0=1.0, v0=0.0, t_min=0, t_max=36, dt=0.01):
    def dynamics(t, y):
        x, v = y
        _, force = potential_func(x, **params)
        return [v, force]

    t_span = (t_min, t_max)
    t_eval = np.arange(t_min, t_max, dt)
    sol = solve_ivp(dynamics, t_span, [x0, v0], t_eval=t_eval)
    return sol.t, sol.y[0], sol.y[1]


# Plot the potential and phase space
def plot_potential_phase_space(potential_case="Dual Harmonic", k=1.0,a = 1.0):
    clear_output(wait=True)
    x = np.linspace(-50, 50, 500)
    
    potential, _ = dual_harmonic_potential(x, k, a)
    params = {'k': k, 'a': a}
    potential_func = dual_harmonic_potential

    t, x_vals, v_vals = simulate_motion(potential_func, params)
    
    # Plotting
    fig, ax = plt.subplots(1, 2, figsize=(12, 5))
    
    # Plot potential
    ax[0].plot(x, np.log(1 + potential), label="Potential")
    ax[0].set_xlabel("x")
    ax[0].set_ylabel("Potential")
    ax[0].set_title(f"{potential_case} Potential")
    ax[0].grid(True)
    ax[0].legend()

    # Plot phase space
    ax[1].plot(x_vals, v_vals, label="Phase Space")
    ax[1].set_xlabel("x")
    ax[1].set_ylabel("v")
    ax[1].set_title("Phase Space")
    ax[1].grid(True)
    ax[1].legend()
    
    # Seaborn KDE plot
    fig, ax = plt.subplots(figsize=(8, 6))
    sns.kdeplot(x=x_vals, y=v_vals, ax=ax, cmap="viridis", fill=True)
    ax.set_xlabel("x")
    ax.set_ylabel("v")
    ax.set_title("Phase Space - KDE Plot")
    plt.tight_layout()
    plt.show()



In [23]:
interact(
    plot_potential_phase_space,
    potential_case=["Dual Harmonic"],
    k=FloatText(value=10.0, description="k (Harmonic)"),
    a=FloatText(value=2.0, description="a (Dual Hump)"),
    
)

interactive(children=(Dropdown(description='potential_case', options=('Dual Harmonic',), value='Dual Harmonic'…

<function __main__.plot_potential_phase_space(potential_case='Dual Harmonic', k=1.0, a=1.0)>